## Data upload using foles from colab

In [0]:
from google.colab import files

In [0]:
!rm 'test - Copy.csv'

rm: cannot remove 'test - Copy.csv': No such file or directory


In [0]:
train_data = list(files.upload().values())[0]

Saving train.csv to train.csv


In [0]:
test_data = list(files.upload().values())[0]

Saving test - Copy.csv to test - Copy.csv


In [0]:
import pandas as pd
import io

In [0]:
train = pd.read_csv(io.StringIO(train_data.decode('utf-8')))
test = pd.read_csv(io.StringIO(test_data.decode('utf-8')))

In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## BernoulliNaiveBayes implemtation

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb 18 11:15:56 2020

@author: parfa
"""


import math
import numpy as np
import statistics 
from collections import Counter

class BernoulliNaiveBayes:

    def __init__(self):
        self.data_dict = {}
        self.stats_analysis = {}
        self.total_row = 0
        
    def fit(self, dataset, targets):
        self._group_class(dataset, targets)
        self._generate_model()
    
    def _group_class(self, dataset, targets):

        assert (targets.shape[0] == dataset.shape[0])
        self.total_row = targets.shape[0]
        for sample, target in zip(dataset, targets):
						
            if target not in self.data_dict:

                self.data_dict[target] = []

            self.data_dict[target].append(sample)
   

    def col_statistics(self, dataset):

        row_len = dataset.shape[0]
        #
        return [(sum(col) + 1)/(row_len + len(Counter(col)) * 1) for col in zip(*dataset)]

    def class_statistics(self):
        
        for class_label, rows in self.data_dict.items():
            class_data = np.vstack(rows)
            self.stats_analysis[class_label] = {
                    'class_prob': class_data.shape[0]/self.total_row,
                    #'class_prob': (class_data.shape[0] + 1)/(self.total_row + (len(self.data_dict) * self.lambda),
                                                             
                    'col_prob': self.col_statistics(np.vstack(rows))
                    }
        # dataset = [np.vstack(value) for key, value in self.data_dict.items()]
        # dataset = np.vstack(data)

    def _generate_model(self):
	    self.class_statistics()


    def calculate_probability(self, x,col_theta1, col_theta0):

        if col_theta1== 0 or col_theta0 == 0:
            return np.zeros((x.shape[0],1))
        
        return (x * math.log(col_theta1/col_theta0)) + ((1 - x) * math.log((1 - col_theta1) / (1 - col_theta0)))

            
    
    # Calculate the probabilities of predicting each class for a given row

    def calculate_class_probabilities(self, rows):

        probabilities = np.zeros((rows.shape[0],1))
        for i, (prob1, prob2) in enumerate(zip(self.stats_analysis[1]['col_prob'], self.stats_analysis[0]['col_prob'])):
            probabilities = probabilities + self.calculate_probability(rows[:,i:i+1], prob1, prob2)
        
        theta1 = self.stats_analysis[1]['class_prob']
        
        probabilities += math.log(theta1 / (1 - theta1))

        return probabilities
        

    def predict(self,x):
        probabilities = self.calculate_class_probabilities(x)
        probabilities[probabilities > 0] = 1
        probabilities[probabilities <= 0] = 0

        
        return probabilities

    def accu_eval(self, predicted, true_value):
        accur = np.zeros(predicted.shape)
        accur[predicted == true_value] = 1
        return (sum(accur) / accur.shape[0]) * 100


In [0]:
bnb = BernoulliNaiveBayes()

In [0]:
result = bag_of_word(io.StringIO(train_data.decode('utf-8')),'review','sentiment',bnb,0.7)

## Data preprocessing

In [0]:
#Followed this tutorial
#https://pythonhealthcare.org/2018/12/15/104-using-free-text-for-classification-bag-of-words/

# import nltk
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
# import model as mdl
import sklearn
from sklearn.metrics import accuracy_score
from sklearn import model_selection
#if not previously performed:
# nltk.download('stopwords')

def bag_of_word2(csv_file,x_col_name,y_col_name,classifer,split_precentage):

    raw_df = pd.read_csv(csv_file)

    x_df = raw_df[x_col_name]
    y_df = raw_df[y_col_name].apply(lambda x: 1 if x == 'positive' else 0)



    stemming = PorterStemmer()
    stops = set(stopwords.words("english"))

    def apply_cleaning_function_to_list(X):
        cleaned_X = []
        for element in X:
            cleaned_X.append(clean_text(element))
            print(element)
        return cleaned_X


    #  This function works on a raw text string, and:
    #     1) changes to lower case
    #     2) tokenizes (breaks down into words
    #     3) removes punctuation and non-word text
    #     4) finds word stems
    #     5) removes stop words
    #     6) rejoins meaningful stem words
    def clean_text(raw_text):
        #convert to lower case
        text = raw_text.lower()

        #tokenize
        print("here")
        tokens = nltk.word_tokenize(text)

        # Keep only words (removes punctuation + numbers)
        # use .isalnum to keep also numbers
        token_words = [w for w in tokens if w.isalpha()]

        #stemming
        stemmed_Words = [stemming.stem(w) for w in token_words]

        #remove stop words
        meaningful_words = [w for w in stemmed_Words if not w in stops]

        #rejoin meaningful stemmed words
        joined_words = (" ".join(meaningful_words))

        #return cleaned data
        return joined_words

    #get text to to clean
    text_to_clean = list(x_df)
    #clean text
    cleaned_text = apply_cleaning_function_to_list(text_to_clean)

    #add cleaned data back into Dataframe
    raw_df['cleaned'+x_col_name] = cleaned_text

    #remove temporary cleaned_text list(after transfer to dataframe)
    del cleaned_text 

    X_train = list(raw_df['cleaned'+x_col_name])
    y_train = list(y_df)

    def create_bag_of_words(X):
        from sklearn.feature_extraction.text import CountVectorizer

        print('Creating bag of words...')
        # Initialize the "CountVectorizer" object, which is scikit-learn's
        # bag of words tool.  
    
        # In this example features may be single words or two consecutive words
        # (as shown by ngram_range = 1,2)

        vectorizer = CountVectorizer(analyzer = "word",   
                                 tokenizer = None,    
                                 preprocessor = None, 
                                 stop_words = None,   
                                 ngram_range = (1,2), 
                                 max_features = 5000
                                ) 
        # fit_transform() does two functions: First, it fits the model
        # and learns the vocabulary; second, it transforms our training data
        # into feature vectors. The input to fit_transform should be a list of 
        # strings. The output is a sparse array
        train_data_feautures = vectorizer.fit_transform(X)

        #convert to numpy array for easy handling
        train_data_feautures = train_data_feautures.toarray()
        #tfidif transform
        from sklearn.feature_extraction.text import TfidfTransformer

        tfidf = TfidfTransformer()
        tfidf_features = tfidf.fit_transform(train_data_feautures).toarray()

        #get words in the vocab
        vocab = vectorizer.get_feature_names()

        return vocab,tfidf_features

    vocab,tfidf_features  = create_bag_of_words(X_train)
    result = (tfidf_features,y_train)
    return result
    




In [0]:
result2 = bag_of_word2(io.StringIO(train_data.decode('utf-8')),'review','sentiment',bnb,0.7)

In [0]:
Y_train_input.shape

(30000,)

In [0]:
X_train_input = result2[0]
Y_train_input = np.array(result2[1])

## Cross validation

In [0]:
from sklearn.model_selection import KFold

kf = model_selection.KFold(n_splits=10, random_state=100)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
accuracy = []

In [0]:
accuracy = []
for train_index, test_index in kf.split(X_train_input):
    bnb_model = BernoulliNaiveBayes()
    X_train, X_test = X_train_input[train_index], X_train_input[test_index]
    y_train, y_test = Y_train_input[train_index], Y_train_input[test_index]
    bnb_model.fit(X_train, y_train)
    accuracy.append(bnb.accu_eval(bnb_model.predict(X_test), np.reshape(y_test, (X_test.shape[0], 1))))

In [0]:
accuracy

[array([83.76666667]),
 array([83.46666667]),
 array([84.16666667]),
 array([83.76666667]),
 array([84.5]),
 array([82.73333333]),
 array([82.56666667]),
 array([84.03333333]),
 array([84.63333333]),
 array([84.63333333])]

In [0]:
sum(accuracy) / 10

array([83.82666667])

In [0]:
'''
The lowest accuracy is 82.56666667 from fold 6
The highest accuracy is 84.63333333 from fold 9 and 10
The average accuracy is 83.82666667
'''